In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, ElasticNet
import statsmodels.api as sm
import statsmodels.formula.api as smf
#from statsmodels.stats.outliers_influence import variance_inflation_factor
#import requests
import json
import time
from Ames_Functions import *
import pickle
pd.set_option("display.max_columns", None)

In [213]:
df_house2021 = load_state_pkl()["df_RE_latlong2021"][0].set_index("SaleID")
df_house2019 = load_state_pkl()["df_RE_latlong2019"][0].set_index("SaleID")
df_biz = pd.read_csv("df_EssentialLatLong_v2.csv")
#df_min_distances = pd.read_csv("df_min_distances.csv", index_col = 0)
df_driv_time_final2019 = load_state_pkl()["2019dfs_for_feature_engineering"][0].filter(regex = "_time")
df_driv_time_final2021 = load_state_pkl()["2021dfs_for_feature_engineering"][0].filter(regex = "_time")

In [215]:
load_state_pkl()["2021dfs_for_feature_engineering"][0].filter(regex = "_time")

,Arts_time,Elem_School_time,Golf_time,Gym_time,High_School_time,Historic_time,ISU_time,Library_time,Medical_time,Movies_time,Organic_Groceries_time,Park_time,Recreation_time,Religion_time,Restaurant_time,Shopping_time,Spa_time,Sports_Venue_time
SaleID,,,,,,,,,,,,,,,,,,
2,633.1,340.3,131.2,288.2,500.7,606.4,518.9,601.5,565.2,777.8,267.9,503.6,513.0,85.5,278.6,421.6,425.7,781.4
3,633.1,340.3,131.2,288.2,500.7,606.4,518.9,601.5,565.2,777.8,267.9,503.6,513.0,85.5,278.6,421.6,425.7,781.4
18,642.9,350.1,141.0,298.0,510.5,616.2,528.7,611.3,575.0,787.6,277.7,513.4,522.8,95.3,288.4,431.4,435.5,791.2
19,646.1,353.3,144.2,301.2,513.7,619.4,531.9,614.5,578.2,790.8,280.9,516.6,526.0,98.5,291.6,434.6,438.7,794.4
20,649.4,356.6,147.5,304.5,517.0,622.7,535.2,617.8,581.5,794.1,284.2,519.9,529.3,101.8,294.9,437.9,442.0,797.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22743,350.0,276.6,318.9,403.4,618.3,391.4,609.7,356.0,374.6,277.0,430.0,398.6,382.9,559.4,305.3,298.6,418.3,473.6
22749,378.6,305.2,347.5,432.0,646.9,420.0,638.3,384.6,403.2,305.6,458.6,427.2,411.5,588.0,333.9,327.2,446.9,502.2
22756,397.4,324.0,366.3,450.8,665.7,438.8,657.1,403.4,422.0,324.4,477.4,446.0,430.3,606.8,352.7,346.0,465.7,521.0


In [ ]:
# df_cross = df_house.join(df_biz, how = "cross", lsuffix = "_house", rsuffix = "_biz")
# df_cross

In [98]:
# df_cross["dist"] = df_cross.apply(lambda x: get_dist(np.array([x.Lat_house, x.Long_house]), np.array([x.Lat_biz, x.Long_biz])), axis = 1)
# #df_cross.to_csv("df_cross.csv")

## Finding differences in two different variables for feature engineering
## (SKIP)

In [439]:
# (df_driv_final["from_time"]
#     .eval("HS_Lib_diff = abs(High_School - Library)")
#     .eval("SV_Rest_diff = abs(Sports_Venue - Restaurant)")
#     .eval("Mov_Rest_diff = abs(Movies-Restaurant)")
#     .eval("Park_Rec_diff = abs(Park - Recreation)")
#     .eval("Groc_Shop_diff = abs(Organic_Groceries - Shopping)")
# ).filter(regex = "diff$").corr()
# #Possible ratios to consider:
# #High_School/Library
# #Sports_Venue/Restaurant
# #Movies/Restaurant
# #Park/Recreation
# #Organic_Groceries/Shopping
# #ISU/Library

,HS_Lib_diff,SV_Rest_diff,Mov_Rest_diff,Park_Rec_diff,Groc_Shop_diff
HS_Lib_diff,1.000000,-0.727417,-0.547363,-0.152308,-0.219560
SV_Rest_diff,-0.727417,1.000000,0.848838,-0.122952,0.397418
Mov_Rest_diff,-0.547363,0.848838,1.000000,-0.274410,0.370700
Park_Rec_diff,-0.152308,-0.122952,-0.274410,1.000000,-0.468057
Groc_Shop_diff,-0.219560,0.397418,0.370700,-0.468057,1.000000


## Finding ratios of distances from two different services for feature engineering 
## (SKIP)

In [652]:
#Possible ratios to consider:
#High_School/Library
#Sports_Venue/Restaurant
#Movies/Restaurant
#Park/Recreation
#Organic_Groceries/Shopping
#ISU/Library

ratio_dict = {}

for a, b in [("High_School", "Library"), ("Sports_Venue", "Restaurant"), ("Movies", "Restaurant"), ("Park", "Recreation"), ("Organic_Groceries", "Shopping"), ("ISU", "Library")]:
      ratio_dict[f"{a}_{b}_ratio"] = (df_driv_final["from_time"]
                                                .apply(lambda x: x[a]/x[b] if x[a] > x[b] else x[b]/x[a], axis = 1)
                                                .rename(f"{a}_{b}_ratio")
                                    )

pd.DataFrame(ratio_dict).join(df_driv_final["from_time"].SalePrice)

,High_School_Library_ratio,Sports_Venue_Restaurant_ratio,Movies_Restaurant_ratio,Park_Recreation_ratio,Organic_Groceries_Shopping_ratio,ISU_Library_ratio,SalePrice
SaleID,,,,,,,
2,1.201318,2.804738,2.791816,1.018666,1.573722,1.159183,78000.0
3,1.201318,2.804738,2.791816,1.018666,1.573722,1.159183,408905.0
18,1.197453,2.743412,2.730929,1.018309,1.553475,1.156232,434066.0
19,1.196223,2.724280,2.711934,1.018196,1.547170,1.155292,88000.0
20,1.194971,2.704985,2.692777,1.018080,1.540816,1.154335,88000.0
...,...,...,...,...,...,...,...
22743,1.736798,1.551261,1.102166,1.041003,1.440054,1.712640,220000.0
22749,1.682007,1.504043,1.092605,1.038153,1.401589,1.659646,217500.0
22756,1.650223,1.477176,1.087238,1.036486,1.379769,1.628904,229000.0


In [ ]:
EDA_report(pd.DataFrame(ratio_dict).drop(["Sports_Venue_Restaurant_ratio", "ISU_Library_ratio"], axis = 1).join(df_driv_final["from_time"].SalePrice))

In [665]:
get_vif(pd.DataFrame(ratio_dict).drop(["Sports_Venue_Restaurant_ratio", "ISU_Library_ratio"], axis = 1))

[2.6474191730790873,
 2.0026501116710853,
 2.9995764720823432,
 1.1216954870182765]

## Lasso feature selection section using lassoCV

### 2021 section

In [ ]:
#checking VIF for all df_driv_final variables
get_vif(df_driv_final["from_time"].iloc[:, :-1])

In [398]:
X_lassoCV = {}
y_lassoCV = {}
X_lassoCV["from_time"] = df_driv_final["from_time"].drop("SalePrice", axis = 1)
y_lassoCV["from_time"] = df_driv_final["from_time"].SalePrice

In [ ]:
lassoCV = LassoCV(normalize = True)
lassoCV.fit(X_lassoCV["from_time"], y_lassoCV["from_time"])
lassoCV.feature_names_in_[np.where(lassoCV.coef_ != 0)]

In [ ]:
#checking VIF for only the lassoCV selected features in df_driv_final
lassoCV_coefs = {}

lassoCV_coefs["from_time"] = df_driv_final["from_time"][lassoCV.feature_names_in_[np.where(lassoCV.coef_ != 0)]]
lassoCV_coefs["from_time"].corr()

#get_vif(lassoCV_coefs["from_time"])

,Lat_house,Movies,Park,Recreation,Spa
Lat_house,1.000000,0.574386,0.114682,0.045949,-0.037722
Movies,0.574386,1.000000,0.563701,0.699062,0.537828
Park,0.114682,0.563701,1.000000,0.541209,0.569881
Recreation,0.045949,0.699062,0.541209,1.000000,0.930184
Spa,-0.037722,0.537828,0.569881,0.930184,1.000000


### 2019 section

In [13]:
#checking VIF for all df_driv_final variables
get_vif(df_driv_time_final2019.iloc[:, 2:-1])

[497.8720803675777,
 166.25262248853497,
 105.49663127559329,
 47.42152644218676,
 165.93510868566523,
 736.8679303864168,
 541.2549557538485,
 474.8168538915485,
 775.8228970286154,
 567.6047378056903,
 74.83131469946107,
 133.6042345276011,
 163.16991644494607,
 47.907261158854055,
 171.76357040297557,
 432.41773327566807,
 243.22226438952976,
 1123.4555166356288]

In [15]:
X_lassoCV = {}
y_lassoCV = {}
X_lassoCV["from_time"] = df_driv_time_final2019.drop("SalePrice", axis = 1)
y_lassoCV["from_time"] = df_driv_time_final2019.SalePrice

In [16]:
lassoCV = LassoCV(normalize = True)
lassoCV.fit(X_lassoCV["from_time"], y_lassoCV["from_time"])
lassoCV.feature_names_in_[np.where(lassoCV.coef_ != 0)]

/Users/lelandmurrin/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(


array(['Golf', 'Library', 'Movies', 'Recreation', 'Religion'],
      dtype=object)

In [17]:
#checking VIF for only the lassoCV selected features in df_driv_final
lassoCV_coefs = {}

lassoCV_coefs["from_time"] = df_driv_time_final2019[lassoCV.feature_names_in_[np.where(lassoCV.coef_ != 0)]]
lassoCV_coefs["from_time"].corr()

#get_vif(lassoCV_coefs["from_time"])

,Golf,Library,Movies,Recreation,Religion
Golf,1.000000,0.233646,0.275352,0.433672,0.505963
Library,0.233646,1.000000,0.791193,0.841805,0.652546
Movies,0.275352,0.791193,1.000000,0.679968,0.396524
Recreation,0.433672,0.841805,0.679968,1.000000,0.747969
Religion,0.505963,0.652546,0.396524,0.747969,1.000000


## Find Homes that had all businesses nearby vs all homes that had businesses far away
## (SKIP)

In [ ]:
#TODO: ADD THIS TO A VISUAL ON AMES MAP

#mean_shop = np.mean(df_driv_final["from_time"][["Shopping"]])[0]
df_nearby = (df_driv_final["from_time"]
    .drop(["Lat_house", "Long_house", "SalePrice"], axis = 1)[df_driv_final["from_time"].drop(["Lat_house", "Long_house", "SalePrice"], axis = 1).transform(lambda x: x < x.mean())]
    .dropna()
    #.isnull()
    #.sum(axis = 0)
    #.sort_values(ascending = False)
) #.plot.hist(alpha = 0.3) #.corr()

df_far = (df_driv_final["from_time"]
    .drop(["Lat_house", "Long_house", "SalePrice"], axis = 1)[df_driv_final["from_time"].drop(["Lat_house", "Long_house", "SalePrice"], axis = 1).transform(lambda x: x > x.mean())]
    .dropna()
    #.isnull()
    #.sum(axis = 0)
    #.sort_values(ascending = False)
) #.plot.hist(alpha = 0.3) #.corr()
df_far.corr().Library #Library, Elem, 

In [ ]:
get_vif(df_far[["Library", "Historic"]])

## Creating feature for average distance of closest 5 services to each house record

### 2021 Section

In [238]:
df_closest2021 = {}

for i in [1, 5, 10, 15]:
    df_closest2021[f"Closest_{i}"] = (df_driv_time_final2021
        #.drop(["Lat_house", "Long_house", "SalePrice"], axis = 1)
        .stack()
        .reset_index()
        .set_axis(["SaleID", "Service", "driv_time"], axis = 1)
        .sort_values("driv_time")
        .groupby("SaleID")
        .head(i)
        .sort_values(["SaleID", "driv_time"])
        .groupby("SaleID")
        .driv_time
        .mean()
        #[["driv_time"]]
        #.mean().driv_time
    )

pd.DataFrame(df_closest2021).Closest_5#.join(df_driv_final["from_time"].SalePrice)

SaleID
2        210.28
3        210.28
18       220.08
19       223.28
20       226.58
          ...  
22743    295.28
22749    323.88
22756    342.68
22768    343.58
22888    309.58
Name: Closest_5, Length: 633, dtype: float64

In [877]:
selected_features_dict = {}
selected_features_dict["Closest_5_feature"] = df_closest['Closest_5']

In [ ]:
#Checking EDA report for each closest 1,5,10,15 businesses to a home
for i in [1,5,10,15]:
    EDA_report(pd.DataFrame(df_closest)[[f"Closest_{i}"]].join(df_driv_final["from_time"].SalePrice))

### 2019 Section

In [ ]:
df_closest = {}

for i in [1, 5, 10, 15]:
    df_closest[f"Closest_{i}"] = (df_driv_time_final2019
        .drop(["Lat_house", "Long_house", "SalePrice"], axis = 1)
        .stack()
        .reset_index()
        .set_axis(["SaleID", "Service", "driv_time"], axis = 1)
        .sort_values("driv_time")
        .groupby("SaleID")
        .head(i)
        .sort_values(["SaleID", "driv_time"])
        .groupby("SaleID")
        .driv_time
        .mean()
        #[["driv_time"]]
        #.mean().driv_time
    )

pd.DataFrame(df_closest).sort_values("Closest_1")#.join(df_driv_final["from_time"].SalePrice)

In [ ]:
selected_features_dict = {}
selected_features_dict["Closest_5_feature"] = df_closest['Closest_5']

In [ ]:
#Checking EDA report for each closest 1,5,10,15 businesses to a home
for i in [1,5,10,15]:
    EDA_report(pd.DataFrame(df_closest)[[f"Closest_{i}"]].join(df_driv_time_final2019.SalePrice))

In [148]:
pd.DataFrame(df_closest).Closest_5

SaleID
92       381.68
93       381.68
96       339.06
98       340.26
100      352.76
          ...  
22745    311.08
22747    312.08
22750    324.98
22755    341.58
22797     99.20
Name: Closest_5, Length: 901, dtype: float64

In [150]:
def append_closest (df_input, df_times, s_closest_num):
    df = df_input.copy()
    df_temp = df_times.stack().reset_index().set_axis(["SaleID", "Service", "Closest_Time"], axis = 1).assign(Service = lambda x: x.Service.str.replace("_time", "_closest"))

    df = (df.join(pd.get_dummies(
                            df_temp.loc[df_temp.groupby("SaleID").Closest_Time.idxmin()],
                            columns = ["Service"]
                            )
                    .set_index("SaleID")
                )
            .join(s_closest_num)
        )
    return df

#EDA_report(pd.get_dummies(df_temp.drop("SalePrice", axis = 1), columns = ["Service"]).join(df_temp.pop("SalePrice")))

## Convenience Measurement Section (average proximity, measured in seconds, to all services)

### 2021 Section

In [ ]:
# s_convenience2021 = (df_driv_final["from_time"]
#         .drop(["Lat_house", "Long_house", "SalePrice"], axis = 1)
#         .stack()
#         .reset_index()
#         .set_axis(["SaleID", "Service", "driv_time"], axis = 1)
#         .sort_values("driv_time")
#         .groupby("SaleID")
#         .driv_time
#         .mean()
#         .sort_values()
#         .rename("Convenience_secs")
#         # .groupby("SaleID")
#         # .driv_time
#         # .mean()
#     )

# pd.DataFrame(s_convenience2021).join(df_house2021.SalePrice)

In [ ]:
EDA_report(pd.DataFrame(s_convenience2021).join(pd.DataFrame(df_closest)).join(df_driv_final["from_time"].SalePrice))

In [890]:
get_vif(pd.DataFrame(df_central).join(pd.DataFrame(df_closest["Closest_1"].rename("Closest_1"))))

[7.650401862528, 7.650401862528]

In [ ]:
# selected_features_dict["Closest_all_feature"] = s_convenience2021
# selected_features_dict

### 2019 Section

In [ ]:
# s_convenience2019 = (df_driv_time_final2019
#         #.drop(["Lat_house", "Long_house", "SalePrice"], axis = 1)
#         .stack()
#         .reset_index()
#         .set_axis(["SaleID", "Service", "driv_time"], axis = 1)
#         .sort_values("driv_time")
#         .groupby("SaleID")
#         .driv_time
#         .mean()
#         .sort_values()
#         .rename("Convenience_secs")
#         # .groupby("SaleID")
#         # .driv_time
#         # .mean()
#     )

# pd.DataFrame(s_convenience2019).join(df_house2019.SalePrice)

In [ ]:
EDA_report(pd.DataFrame(s_convenience2019).join(pd.DataFrame(df_closest)).join(df_house2019.SalePrice))

In [ ]:
get_vif(pd.DataFrame(s_convenience2019).join(pd.DataFrame(df_closest["Closest_1"].rename("Closest_1"))))

In [ ]:
# selected_features_dict["Closest_all_feature"] = s_convenience2019
# selected_features_dict

In [ ]:
#Looking at which services are on average farthest away from all houses
#TODO: map this out
df_driv_time_final2019.iloc[:,2:-1].mean(axis = 0).sort_values(ascending = False)

In [188]:
def add_convenience_series (df_input, df_driv_time_final):
    df = df_input.copy()
    s_convenience = (df_driv_time_final
        #.drop(["Lat_house", "Long_house", "SalePrice"], axis = 1)
        .stack()
        .reset_index()
        .set_axis(["SaleID", "Service", "driv_time"], axis = 1)
        .sort_values("driv_time")
        .groupby("SaleID")
        .driv_time
        .mean()
        .sort_values()
        .rename("Convenience_secs")
        .apply(round, ndigits = 1)
        # .groupby("SaleID")
        # .driv_time
        # .mean()
    )
    return df.join(s_convenience)

## Centrality Measurement Section (Proximity to Historic Service which contains City Hall and Old Town Historic District)

In [910]:
selected_features_dict["Central_historic_feature"] = df_driv_final["from_time"].Historic

dict_keys(['Closest_5_feature', 'Closest_all_feature', 'Central_historic_feature'])

In [176]:
df_driv_time_final2019.Historic_time

SaleID
92       626.9
93       626.9
96       524.0
98       525.2
100      537.7
         ...  
22745    407.2
22747    408.2
22750    421.1
22755    437.7
22797    135.6
Name: Historic_time, Length: 901, dtype: float64

In [177]:
def add_centrality_series (df_input, df_driv_time_final):
    df = df_input.copy()
    s_centrality = df_driv_time_final.Historic_time.rename("Central_historic_feature")

    return df.join(s_centrality)

## Count Number of Businesses within 1 minute of each house section

### 2021 Section

In [246]:
num_biz_dict = {}

for i in np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])*60:
    num_biz_dict[f"biz_{i}_secs"] = (df_driv_time_final2021
        #.drop(["Lat_house", "Long_house", "SalePrice"], axis = 1)
        .stack()
        .reset_index()
        .set_axis(["SaleID", "Service", "driv_time"], axis = 1)
        .groupby("SaleID")
        .driv_time
        .apply(lambda x: (x <= i).sum())
        .rename("count")
        #.where(lambda x: x <= i)
        # .query(f"driv_time <= {i}")
        # .SaleID
        # .value_counts()
        #.sort_values("driv_time")
        #.filter(lambda x: x.driv_time <= i)
        # .head(i)
        # .sort_values(["SaleID", "driv_time"])
        # .groupby("SaleID")
        # .driv_time
        # .mean()
    )

pd.DataFrame(num_biz_dict).join(df_house2021.SalePrice)

,biz_60_secs,biz_120_secs,biz_180_secs,biz_240_secs,biz_300_secs,biz_360_secs,biz_420_secs,biz_480_secs,biz_540_secs,biz_600_secs,SalePrice
SaleID,,,,,,,,,,,
2,0,1,2,2,5,6,6,8,12,13,78000.0
3,0,1,2,2,5,6,6,8,12,13,408905.0
18,0,1,2,2,5,6,6,8,12,13,434066.0
19,0,1,2,2,4,6,6,8,12,13,88000.0
20,0,1,2,2,4,6,6,8,12,13,88000.0
...,...,...,...,...,...,...,...,...,...,...,...
22743,0,0,0,0,3,7,13,15,15,16,220000.0
22749,0,0,0,0,0,5,10,14,15,16,217500.0
22756,0,0,0,0,0,4,7,14,15,15,229000.0


In [ ]:
EDA_report(pd.DataFrame(num_biz_dict).join(df_driv_final["from_time"].SalePrice))

In [ ]:
pd.DataFrame(num_biz_dict).biz_480_secs.plot.hist()

In [ ]:
get_vif(pd.DataFrame(num_biz_dict)[["biz_180_secs", "biz_480_secs"]])

In [ ]:
selected_features_dict["num_biz_180s_feature"], selected_features_dict["num_biz_480s_feature"] = num_biz_dict["biz_180_secs"], num_biz_dict["biz_480_secs"]
selected_features_dict.keys()

In [740]:
#mean driving time in minutes for all businesses
df_driv_final["from_time"].drop(["Lat_house", "Long_house", "SalePrice"], axis = 1).describe().T['mean'].mean()/60

5.944335030132818

In [937]:
pickle.dump(selected_features_dict, open("selected_features_dict.pkl", 'wb'))

### 2019 Section

In [ ]:
num_biz_dict = {}

for i in np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])*60:
    num_biz_dict[f"biz_{i}_secs"] = (df_driv_time_final2019
        .drop(["Lat_house", "Long_house", "SalePrice"], axis = 1)
        .stack()
        .reset_index()
        .set_axis(["SaleID", "Service", "driv_time"], axis = 1)
        .groupby("SaleID")
        .driv_time
        .apply(lambda x: (x <= i).sum())
        .rename("count")
        #.where(lambda x: x <= i)
        # .query(f"driv_time <= {i}")
        # .SaleID
        # .value_counts()
        #.sort_values("driv_time")
        #.filter(lambda x: x.driv_time <= i)
        # .head(i)
        # .sort_values(["SaleID", "driv_time"])
        # .groupby("SaleID")
        # .driv_time
        # .mean()
    )

pd.DataFrame(num_biz_dict).join(df_driv_time_final2019.SalePrice)

In [ ]:
EDA_report(pd.DataFrame(num_biz_dict).join(df_driv_time_final2019.SalePrice))

In [ ]:
pd.DataFrame(num_biz_dict).biz_480_secs.plot.hist()

In [63]:
get_vif(pd.DataFrame(num_biz_dict)[["biz_180_secs", "biz_480_secs"]])

[2.493339804500264, 2.493339804500263]

In [ ]:
selected_features_dict["num_biz_180s_feature"], selected_features_dict["num_biz_480s_feature"] = num_biz_dict["biz_180_secs"], num_biz_dict["biz_480_secs"]
selected_features_dict.keys()

dict_keys(['Closest_5_feature', 'Closest_all_feature', 'Central_historic_feature', 'num_biz_180s_feature', 'num_biz_480s_feature'])

In [ ]:
#mean driving time in minutes for all businesses
df_driv_final["from_time"].drop(["Lat_house", "Long_house", "SalePrice"], axis = 1).describe().T['mean'].mean()/60

5.944335030132818

In [240]:
def add_num_biz_by_time (df_input, df_driv_time_final, times):
        df = df_input.copy()
        for t in times: 
                num_biz = (df_driv_time_final
                #.drop(["Lat_house", "Long_house", "SalePrice"], axis = 1)
                .stack()
                .reset_index()
                .set_axis(["SaleID", "Service", "driv_time"], axis = 1)
                .groupby("SaleID")
                .driv_time
                .apply(lambda x: (x <= t).sum())
                .rename(f"biz_{t}_secs")
                )
                df = df.join(num_biz)
        return df

## Count the number of houses around each service

In [826]:
num_house_dict = {}

for s in np.array([2, 3, 4])*60:
    num_house_dict[f"house_{s}_secs"] = (df_driv_final["from_time"]
        .drop(["Lat_house", "Long_house", "SalePrice"], axis = 1)
        .stack()
        .reset_index()
        .set_axis(["SaleID", "Service", "driv_time_secs"], axis = 1)
        .groupby('Service')
        .apply(lambda x: x[x.driv_time_secs < s].driv_time_secs.count())
    )

pd.DataFrame(num_house_dict).join(df_biz.Service.value_counts().rename("Service_count")).apply(lambda x: round(x/x.Service_count, 2), axis = 1).drop("Service_count", axis = 1)

,house_120_secs,house_180_secs,house_240_secs
Service,,,
Arts,17.67,43.00,64.33
Elem_School,23.50,49.50,111.00
Golf,1.33,16.33,33.00
Gym,27.25,52.75,81.00
High_School,2.00,28.00,110.00
Historic,29.00,58.00,80.50
ISU,0.00,3.00,64.00
Library,38.00,81.00,114.00
Medical,74.00,129.00,175.00


In [813]:
mean_SalePrice = {}

for s in np.array([2, 3, 4])*60:
    mean_SalePrice[f"mean_salePrice_{s}_secs"] = (df_driv_final["from_time"]
        .drop(["Lat_house", "Long_house"], axis = 1)
        .set_index("SalePrice")
        .stack()
        .reset_index()
        .set_axis(["SalePrice", "Service", "driv_time_secs"], axis = 1)
        .groupby('Service')
        .apply(lambda x: x[x.driv_time_secs < s].SalePrice.mean())
        .transform(round, ndigits = 2)
    )

pd.DataFrame(mean_SalePrice).sort_values(by = "mean_salePrice_120_secs")

,mean_salePrice_120_secs,mean_salePrice_180_secs,mean_salePrice_240_secs
Service,,,
High_School,147375.00,213282.14,206375.00
Historic,166041.12,184953.67,196863.83
Medical,178680.08,186369.58,200287.29
Recreation,184346.26,188721.04,202233.86
Spa,191721.71,208934.29,212455.78
Organic_Groceries,202259.42,212034.15,240566.40
Shopping,204111.48,200157.49,208839.61
Library,215477.43,204664.55,207509.27
Arts,218552.83,202680.42,213048.71


In [832]:
df_nearby_houses = (pd.DataFrame(mean_SalePrice)
    .join(pd.DataFrame(num_house_dict)
          .join(df_biz.Service.value_counts().rename("Service_count"))
          .apply(lambda x: round(x/x.Service_count, 2), axis = 1)
          .drop("Service_count", axis = 1))
    .sort_values('mean_salePrice_120_secs')
)

In [837]:
get_vif(df_nearby_houses.dropna()[["mean_salePrice_120_secs", "house_120_secs"]])

[1.697142040569102, 1.697142040569102]

In [ ]:
df_biz.Service.value_counts().rename("Service_count")

## Real Estate Dataset Feature Engineering

Adding fields: TotalArea and TotalArea_Gar

In [ ]:
# df_house = df_house.eval("TotalArea = GLA + TtlBsmtSF	").eval("TotalArea_Gar = GLA + TtlBsmtSF	+ GarageArea")
# #df_RE_latlong.eval("TotalArea_Gar = GLA + TtlBsmtSF	+ GarageArea_RE", inplace = True)
# df_house#.filter(regex = "Area|SF|GLA")

Adding Fields "Has_Fireplace" and "Has_Garage" and "Has_Bsmt"

In [35]:
# has_feature = lambda x: 1 if x > 0 else 0

In [52]:
# df_house["Has_Fireplace"] = df_house.Fireplaces.apply(has_feature)

In [47]:
# df_house["Has_Garage"] = df_house.GarageArea.apply(has_feature)

In [46]:
# df_house["Has_Bsmt"] = df_house.TtlBsmtSF.apply(has_feature)

Adding Field TotalExt and dummy variables for it

In [114]:
# df_house2019.Ext1.unique().shape, df_house2019.Ext2.unique().shape, df_house2021.Ext1.unique().shape, df_house2021.Ext2.unique().shape

((13,), (16,), (12,), (14,))

In [80]:
# df_Ext_dummies = pd.get_dummies(df_house.Ext2).rename(columns = {"Brk Cmn": "BrkComm", "CmentBd": "CemntBd", "Wd Shng": "WdShing"})

# for col in df_house.Ext1.unique():
#     df_Ext_dummies[[col]] += pd.get_dummies(df_house.Ext1)[[col]]

# df_Ext_dummies.rename(columns = {"C'Block": "CBlock"})
# df_Ext_dummies.rename(columns = {col:col + "_Ext" for col in df_Ext_dummies.columns}, inplace = True)
# #(pd.get_dummies(df_house.Ext1)[["AsbShng"]] + pd.get_dummies(df_house.Ext2)[["AsbShng"]]).sum()

In [84]:
# df_house = df_house.join(df_Ext_dummies)

Adding dummy variable columns for GarageType and HouseStyle

In [100]:
# clean_colname = lambda x: x.replace("-", "_").replace(".", "_").replace("/", "_").replace(" ", "_")

In [ ]:
# df_house = df_house.join(pd.get_dummies(df_house.GarageType).rename(columns = {col:clean_colname(col) + "_GrType" for col in pd.get_dummies(df_house.GarageType).columns}))

In [98]:
# "2-h.5 /house".replace("-", "_").replace(".", "_").replace("/", "_").replace(" ", "_")

'2_h_5__house'

In [103]:
# df_house = df_house.join(pd.get_dummies(df_house.HouseStyle).rename(columns = {col:clean_colname(col) + "_HsStyle" for col in pd.get_dummies(df_house.HouseStyle).columns}))

In [104]:
# df_house.to_csv("df_RE_latlongs_v3.csv")

In [96]:
def transform_RE_features (df_input):
    df_RE = df_input.copy()

    # Adding Fields "Has_Fireplace" and "Has_Garage" and "Has_Bsmt"
    has_feature = lambda x: 1 if x > 0 else 0
    df_RE["Has_Fireplace"] = df_RE.Fireplaces.apply(has_feature)
    df_RE["Has_Garage"] = df_RE.GarageArea.apply(has_feature)
    df_RE["Has_Bsmt"] = df_RE.TtlBsmtSF.apply(has_feature)

    # Adding fields: TotalArea and TotalArea_Gar
    df_RE = df_RE.eval("TotalArea = GLA + TtlBsmtSF	").eval("TotalArea_Gar = GLA + TtlBsmtSF	+ GarageArea")
    
    # Adding Field TotalExt and dummy variables for it
    df_Ext_dummies = pd.get_dummies(df_RE.Ext2).rename(columns = {"Brk Cmn": "BrkComm", "CmentBd": "CemntBd", "Wd Shng": "WdShing"})

    for col in df_RE.Ext1.unique():
        df_Ext_dummies[[col]] += pd.get_dummies(df_RE.Ext1)[[col]]

    df_Ext_dummies.rename(columns = {"C'Block": "CBlock"}, inplace = True)
    df_Ext_dummies.rename(columns = {col:col + "_Ext" for col in df_Ext_dummies.columns}, inplace = True)
    df_RE = df_RE.join(df_Ext_dummies)

    # Adding dummy variable columns for GarageType
    clean_colname = lambda x: x.replace("-", "_").replace(".", "_").replace("/", "_").replace(" ", "_")
    df_RE = df_RE.join(pd.get_dummies(df_RE.GarageType).rename(columns = {col:clean_colname(col) + "_GrType" for col in pd.get_dummies(df_RE.GarageType).columns}))

    # Adding dummy variable columns for HouseStyle and renaming leading numbers with words
    df_RE = df_RE.join(pd.get_dummies(df_RE.HouseStyle).rename(columns = {col:clean_colname(col) + "_HsStyle" for col in pd.get_dummies(df_RE.HouseStyle).columns}))
    df_RE.rename(columns = {col: col.replace(" ", "_").replace("1", "One").replace("2", "Two").replace("5", "Five") for col in df_RE.columns}, inplace = True)

    return df_RE

In [282]:
 #For diagnostic purposes
def Ext_test (df_house):
    df_Ext_dummies = pd.get_dummies(df_house.Ext2).rename(columns = {"Brk Cmn": "BrkComm", "CmentBd": "CemntBd", "Wd Shng": "WdShing"})

    for col in df_house.Ext1.unique():
        df_Ext_dummies[[col]] += pd.get_dummies(df_house.Ext1)[[col]]

    df_Ext_dummies.rename(columns = {"C'Block": "CBlock"}, inplace = True)
    df_Ext_dummies.rename(columns = {col:col + "_Ext" for col in df_Ext_dummies.columns}, inplace = True)
    return df_Ext_dummies.columns

display(Ext_test(df_house2019))
display(Ext_test(df_house2021))
display(set(Ext_test(df_house2019)) - (set(Ext_test(df_house2021))))
set(Ext_test(df_house2021)) - (set(Ext_test(df_house2019)))


Index(['AsbShng_Ext', 'AsphShn_Ext', 'BrkComm_Ext', 'BrkFace_Ext',
       'CBlock_Ext', 'CemntBd_Ext', 'HdBoard_Ext', 'ImStucc_Ext',
       'MetalSd_Ext', 'No Data_Ext', 'Other_Ext', 'Plywood_Ext', 'Stucco_Ext',
       'VinylSd_Ext', 'Wd Sdng_Ext', 'WdShing_Ext'],
      dtype='object')

Index(['AsbShng_Ext', 'BrkComm_Ext', 'BrkFace_Ext', 'CemntBd_Ext',
       'HdBoard_Ext', 'ImStucc_Ext', 'MetalSd_Ext', 'No Data_Ext',
       'Plywood_Ext', 'Stone_Ext', 'Stucco_Ext', 'VinylSd_Ext', 'Wd Sdng_Ext',
       'WdShing_Ext'],
      dtype='object')

{'AsphShn_Ext', 'CBlock_Ext', 'Other_Ext'}

{'Stone_Ext'}

In [318]:
#Preparing df_feature_all for entry into pickle file for 2019, 2021 and both combined into one master dataset (df_feature_all)
df_feature_all2019 = (df_house2019
    .pipe(transform_RE_features)
    .filter(regex = "Yr|_Ext|Has|Area|SF|Cars|Fireplaces|GrType|HsStyle").drop(["YrSold_YYYY"], axis = 1)
    .pipe(append_closest, df_driv_time_final2019, pd.DataFrame(df_closest).Closest_5)
    .pipe(add_convenience_series, df_driv_time_final2019)
    .pipe(add_centrality_series, df_driv_time_final2019)
    .pipe(add_num_biz_by_time, df_driv_time_final2019, [180, 480])
    #.isna().any().sum()
)

df_feature_all2021 = (df_house2021
    .pipe(transform_RE_features)
    .filter(regex = "Yr|_Ext|Has|Area|SF|Cars|Fireplaces|GrType|HsStyle").drop(["YrSold_YYYY"], axis = 1)
    .pipe(append_closest, df_driv_time_final2021, pd.DataFrame(df_closest2021).Closest_5)
    .pipe(add_convenience_series, df_driv_time_final2021)
    .pipe(add_centrality_series, df_driv_time_final2021)
    .pipe(add_num_biz_by_time, df_driv_time_final2021, [180, 480])
)

df_feature_all = df_feature_all2019.eval("Year = 2019").append(df_feature_all2021.eval("Year = 2021")).fillna(0)

/var/folders/2n/5k95xl754cb_szsgjkmj33k80000gn/T/ipykernel_12280/3664117225.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_feature_all = df_feature_all2019.eval("Year = 2019").append(df_feature_all2021.eval("Year = 2021")).fillna(0)


In [321]:
desc = "post feature engineering including boolean fields for Fireplace/Gar/Bsmt, dummy fields for Ext, GrType and HouseStyle, TotalArea/TotalArea_Gar fields, and drive time related fields (Closest_5, Convenience etc)"

save_state_pkl("df_house2021_after_feature_engineering",
                df_feature_all2021,
                "df_house2021 " + desc
                )

save_state_pkl("df_house2019_after_feature_engineering",
                df_feature_all2019,
                "df_house2019 " + desc
                )

save_state_pkl("df_house_both_after_feature_engineering",
                df_feature_all,
                "df_house2019 + df_house2021 " + desc
                )